# **GTSRB Project**

In that project I will write a model that can predict name of the traffic sign. It knows 43 signs. I will use a Tensorflow Keras CNN model to train it.
You can find the data in [Kaggle](https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign). Let's start.

In [ ]:
#Importing libraries
import pandas as pd
import os
import numpy as np
import tensorflow as tf

In [ ]:
#Importing my drive
#Before the project I imported the data in Google Drive from Kaggle for use Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Paths
train_path = "/content/drive/MyDrive/Colab Notebooks/train/"
test_path = "/content/drive/MyDrive/Colab Notebooks/test/"

In [ ]:
#Labels
#In dataset classes seperated with numbers didn't with their names
encoded_labels=["0","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","41","42"]

In [ ]:
#Train  Data
img_list=[]
label_list=[]
for label in encoded_labels:
    for img_file in os.listdir(train_path+label):
        img_list.append(train_path+label+'/'+img_file)
        label_list.append(label)

In [ ]:
#Converting into a dataset
train=pd.DataFrame({'img':img_list,'label':label_list})

In [ ]:
train.head()

,img,label
0,/content/drive/MyDrive/Colab Notebooks/train/0...,0
1,/content/drive/MyDrive/Colab Notebooks/train/0...,0
2,/content/drive/MyDrive/Colab Notebooks/train/0...,0
3,/content/drive/MyDrive/Colab Notebooks/train/0...,0
4,/content/drive/MyDrive/Colab Notebooks/train/0...,0


In [ ]:
#Test Data
#Test data didn't encode. So I read the information dataset. In that dataset, there are paths an encodes of test data.
test=pd.read_csv("Test.csv")
test.head()

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,53,54,6,5,48,49,16,Test/00000.png
1,42,45,5,5,36,40,1,Test/00001.png
2,48,52,6,6,43,47,38,Test/00002.png
3,27,29,5,5,22,24,33,Test/00003.png
4,60,57,5,5,55,52,11,Test/00004.png


In [ ]:
#I need only Path and ClassId
test=test[["Path","ClassId"]]

In [ ]:
#I am making Path information lower to read
test["Path"]=test["Path"].str.lower()

In [ ]:
#I am adding true path way to path column
test["Path"]="/content/drive/MyDrive/Colab Notebooks/"+test["Path"]

In [ ]:
#Renaming to concat my data with train data
test=test.rename(columns={"Path":"img"})
test=test.rename(columns={"ClassId":"label"})

In [ ]:
#Concat train and test
#That is my master dataset
df=pd.concat([train,test],axis=0)

In [ ]:
df.head()

,img,label
0,/content/drive/MyDrive/Colab Notebooks/train/0...,0
1,/content/drive/MyDrive/Colab Notebooks/train/0...,0
2,/content/drive/MyDrive/Colab Notebooks/train/0...,0
3,/content/drive/MyDrive/Colab Notebooks/train/0...,0
4,/content/drive/MyDrive/Colab Notebooks/train/0...,0


In [ ]:
df.tail()

,img,label
12625,/content/drive/MyDrive/Colab Notebooks/test/12...,12
12626,/content/drive/MyDrive/Colab Notebooks/test/12...,33
12627,/content/drive/MyDrive/Colab Notebooks/test/12...,6
12628,/content/drive/MyDrive/Colab Notebooks/test/12...,7
12629,/content/drive/MyDrive/Colab Notebooks/test/12...,10


In [ ]:
df.isnull().sum()

img      0
label    0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51839 entries, 0 to 12629
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   img     51839 non-null  object
 1   label   51839 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB


In [ ]:
#My labels are numbers but above write they are strings. Why?
#Because path ways can olny be strings
#Thats why I need to convert them to integers
df["label"]=df["label"].astype(int)

In [ ]:
#I want to know the names of my labels
#That is necessary for my Streamlit app too
#I am making a dictionary to get the name
d = {       0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)',
            2:'Speed limit (50km/h)',
            3:'Speed limit (60km/h)',
            4:'Speed limit (70km/h)',
            5:'Speed limit (80km/h)',
            6:'End of speed limit (80km/h)',
            7:'Speed limit (100km/h)',
            8:'Speed limit (120km/h)',
            9:'No passing',
            10:'No passing veh over 3.5 tons',
            11:'Right-of-way at intersection',
            12:'Priority road',
            13:'Yield',
            14:'Stop',
            15:'No vehicles',
            16:'Veh > 3.5 tons prohibited',
            17:'No entry',
            18:'General caution',
            19:'Dangerous curve left',
            20:'Dangerous curve right',
            21:'Double curve',
            22:'Bumpy road',
            23:'Slippery road',
            24:'Road narrows on the right',
            25:'Road work',
            26:'Traffic signals',
            27:'Pedestrians',
            28:'Children crossing',
            29:'Bicycles crossing',
            30:'Beware of ice/snow',
            31:'Wild animals crossing',
            32:'End speed + passing limits',
            33:'Turn right ahead',
            34:'Turn left ahead',
            35:'Ahead only',
            36:'Go straight or right',
            37:'Go straight or left',
            38:'Keep right',
            39:'Keep left',
            40:'Roundabout mandatory',
            41:'End of no passing',
            42:'End no passing veh > 3.5 tons' }

In [ ]:
#I am adding name column
#In this code, the map() function is used to apply a dictionary transformation to the values in the "label" column of a pandas DataFrame. Each value in the "label" column is replaced with its corresponding value from the dictionary, and the results are stored in a new column named "name".
df["name"]=df["label"].map(d)

In [ ]:
df.isnull().sum()

img      0
label    0
name     0
dtype: int64

In [ ]:
df.shape

(51839, 3)

In [ ]:
#Image Preprocessing
import cv2
x=[]
for img in df["img"]:
    img=cv2.imread(str(img)) #That read the image
    img=cv2.resize(img,(170,170)) #That resize the image to 170x170
    img=img/255.0 #That normalize the image-->Normalizing makes the rgb values between 0 and 1. That facilitates our computers to process
    x.append(img)

In [ ]:
#I should make x to an array for keras
x=np.array(x)

In [ ]:
#My target column
y=df["label"]

In [ ]:
#Train Test Split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [ ]:
#Importing Keras
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, Input, MaxPooling2D, Dropout, BatchNormalization

In [ ]:
#I am making my model
#I am using sequential model
#Input layer with a shape of (170, 170, 3), indicating an input image size of 170x170 pixels with 3 color channels (RGB).
#Convolutional layer with 32 filters, each with a kernel size of (3,3) and ReLU activation.
#MaxPooling layer with a pool size of (2,2) for downsampling.
#Additional convolutional layers followed by max pooling layers with increasing numbers of filters (64, 90, 110, 115).
#Flatten layer to convert the 2D feature maps into a 1D vector.
#Fully connected (Dense) layer with 128 neurons.
#Output layer with 43 neurons (assuming it's a classification task with 43 classes) and softmax activation, representing the probabilities of each class.
#The model is compiled using the Adam optimizer, sparse categorical cross-entropy loss function, and accuracy as the metric to monitor during training.
model=Sequential()
model.add(Input(shape=(170,170,3)))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(90,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(110,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(115,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128))
model.add(Dense(43, activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
#I am training my model
history=model.fit(x_train,y_train,epochs=20,validation_data=(x_test,y_test))

Epoch 1/20
1296/1296 [==============================] - 255s 196ms/step - loss: 0.7038 - accuracy: 0.7980 - val_loss: 0.1364 - val_accuracy: 0.9596
Epoch 2/20
1296/1296 [==============================] - 247s 191ms/step - loss: 0.0843 - accuracy: 0.9773 - val_loss: 0.0754 - val_accuracy: 0.9818
Epoch 3/20
1296/1296 [==============================] - 248s 191ms/step - loss: 0.0528 - accuracy: 0.9848 - val_loss: 0.0916 - val_accuracy: 0.9742
Epoch 4/20
1296/1296 [==============================] - 249s 192ms/step - loss: 0.0395 - accuracy: 0.9888 - val_loss: 0.0410 - val_accuracy: 0.9915
Epoch 5/20
1296/1296 [==============================] - 247s 191ms/step - loss: 0.0336 - accuracy: 0.9901 - val_loss: 0.0464 - val_accuracy: 0.9887
Epoch 6/20
1296/1296 [==============================] - 250s 193ms/step - loss: 0.0297 - accuracy: 0.9916 - val_loss: 0.0602 - val_accuracy: 0.9870
Epoch 7/20
1296/1296 [==============================] - 247s 190ms/step - loss: 0.0274 - accuracy: 0.9916 - val_

In [ ]:
#I am saving my model for making my streamlit app
model.save("my_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


To conclusion I can say my accuracy score is sufficient. If we think in my dataset were 51000 images my accuracy is very good. But in AI projects sometimes metrics don't show the truth. So I suggest you to test your model in real life. For example you can make a Streamlit app like me. Here you can find my [Hugging Face Account](https://huggingface.co/spaces/emirkaanozdemr/traffic-signs)